## Loading everything

In [1]:
%runfile ../zetatypes/util/lazylist.py
%runfile ../zetatypes/ts/monoid.py
%runfile ../zetatypes/ts/ring.py
%runfile ../zetatypes/util/berlekamp.py
%runfile ../zetatypes/ts/complex.py

In [2]:
%runfile ../zetatypes/mf/mf.py
%runfile ../zetatypes/mf/library.py

In [3]:
loadMultiplicativeLibrary("../data/classical-functions.json")
defineTS()
defineMF()

In [4]:
mfd('euler_phi', name='phi')
mfd('sigma_1')
mfd('tau')
mfd('one')
mfd('id')
mfd('id_2')
mfd('id_3')
mfd('zero')
mfd('liouville')
mfd('sigma')
mfd('mu')
print "defined!"

defined!


# Automated Identity explorer

The purpose of this program is to find all identities of or under a specific cost.

## What is an Identity?

In this program, an identity is an equality between two objects of type 'MultiplicativeFunction', meaning a tree-structure based on specified atomic functions, negation, circleplus, circleproduct, and adamsoperations for any n.

## The Cost of an Identity

Let $c(z)$ be the cost of $z$. Since all identities essentially are a tree structure of various operations and 'atomic' functions, we specify the cost based on the tree structure:

$c(f) = \text{evendimension}(f) + \text{odddimension}(f)$ when $f$ is an atomic multiplicative function

$c(f + g) = c(f) + c(g) + 1$

$c(f * g) = c(f) + c(g) + 2$

$c(\psi^n(f)) = n - 1 + c(f) \text{ for } n > 1$

$c(-f) = c(f) + 1$


Finally,

$c(f = g) = c(f) + c(g)$

It is easy to verify that for any $n$, there is a finite number of identities of cost $n$

### The pool class partitions items into classes

In [5]:
class Pool():
    def __init__(self, indexer):
        self.pool = {}
        self.indexer = indexer

    def __iadd__(self, f):
        index = self.indexer(f)
        if index in self.pool:
            self.pool[index].append(f)
        else:
            self.pool[index] = [f]

    def __lshift__(self, by):
        for x in by:
            self < x

    def __lt__(self, by):
        self += by

    def __getitem__(self, index):
        return self.pool[index]

    def __iter__(self):
        for v in self.pool:
            for f in self.pool[v]:
                yield f

    def __contains__(self, index):
        return index in self.pool

    def __str__(self):
        return str(self.pool)

    def __iter__(self):
        for v in self.pool:
            for f in self.pool[v]:
                yield f

    def __contains__(self, index):
        return index in self.pool

    def __str__(self):
        return str(self.pool)

### The following cell contains everything related to cost-based functions

With a few assumptions (for optimization):

* The cost of an atom is determined by itself
* Given a cost, one can form a list of cost-combinations such that all direct sums of cost n are represented by combining expressions of cost given by said cost-combinations
* The same but for tensor product, adamsoperations, and negation

In [6]:
def atomCost(f):
    return operator.add(*f.symbol.superdimension())

def negateCombinations(cost):
    if cost < 1:
        return []
    else:
        return [((True, cost - 1),)]

def adamsCombinations(cost):
    if cost < 1:
        return []
    else:
        return [((False, k + 2), (True, (cost - 1) - k)) for k in range(0, cost)]

def sumCombinations(cost):
    if cost < 1:
        return []
    else:
        return [((True, k), (True, (cost - 1) - k)) for k in range(0, cost)]

def productCombinations(cost):
    if cost < 2:
        return []
    else:
        return [((True, k), (True, (cost - 2) - k)) for k in range(0, cost - 1)]

In [7]:
funcs = [id, one, phi, mu, tau, phi, sigma]

# Sorting by cost
funcsByCost = Pool(atomCost)
funcsByCost << funcs

print funcsByCost

{1: [id, one, mu], 2: [euler_phi, tau, euler_phi, sigma]}


In [8]:
combinators = [
   #(negateCombinations,  lambda x   : -x),
    (adamsCombinations,   lambda n, x: x.adamsoperation(n)),
    (sumCombinations,     lambda x, y: x + y),
    (productCombinations, lambda x, y: x * y),
]

In [9]:
# Upperbound on the cost-loop
boundcost = 10

# Create all pools for each cost, with a symbol-based indexer
pools = [Pool(lambda x: x.symbol) for i in range(boundcost)]



for cost in range(boundcost):
    if cost in funcsByCost:
        pools[cost] << funcsByCost[cost]

    for costFunction, combinator in combinators:
        for costresult in costFunction(cost):
            inputPools = [pools[value] if isCost else [value] for (isCost, value) in costresult]
            for inputs in itertools.product(*inputPools):
                pools[cost] < combinator(*inputs)


In [10]:
identities = []

for lhsCost in range(boundcost):
    for rhsCost in range(0, boundcost + 1 - lhsCost):
        for symb in pools[lhsCost].pool:
            if symb in pools[rhsCost].pool:
                if lhsCost == rhsCost:
                    it = itertools.combinations(pools[lhsCost][symb], r=2)
                else:
                    it = itertools.product(pools[lhsCost][symb], pools[rhsCost][symb])

                for lhs, rhs in it:
                    identities.append(lhs & rhs)

In [11]:
all(x.check() for x in identities)

True

In [12]:
len(identities)

62218

In [13]:
start = 50000 #3284 #1435 #2478

In [14]:
identities[start:start + 10]

[Psi^2 Psi^2 (one * mu) + id = euler_phi,
 Psi^2 Psi^2 (one * mu) + id = euler_phi,
 Psi^2 Psi^3 Psi^2 Psi^2 mu + id = euler_phi,
 Psi^2 Psi^3 Psi^2 Psi^2 mu + id = euler_phi,
 Psi^2 Psi^3 Psi^3 mu + id = euler_phi,
 Psi^2 Psi^3 Psi^3 mu + id = euler_phi,
 Psi^2 Psi^4 Psi^2 mu + id = euler_phi,
 Psi^2 Psi^4 Psi^2 mu + id = euler_phi,
 Psi^2 Psi^5 mu + id = euler_phi,
 Psi^2 Psi^5 mu + id = euler_phi]

In [15]:
html(map(lambda x: '$$' + latex(x) + '$$', identities[start:start + 10]))

[<script type="math/tex; mode=display"> \sum_{d \mid n} \sum_{d_{0} \mid d^2} \sum_{d_{1} \mid \left(\frac{d^2}{d_{0}}\right)^2} \left(1 \cdot \mu\left(\frac{\left(\frac{d^2}{d_{0}}\right)^2}{d_{1}}\right)\right) \cdot \lambda\left(d_{1}\right) \cdot \left(1 \cdot \mu\left(d_{1}\right)\right) \cdot \lambda\left(d_{0}\right) \cdot \sum_{d_{2} \mid \left(d_{0}\right)^2} \left(1 \cdot \mu\left(\frac{\left(d_{0}\right)^2}{d_{2}}\right)\right) \cdot \lambda\left(d_{2}\right) \cdot \left(1 \cdot \mu\left(d_{2}\right)\right) \cdot {\frac{n}{d}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d_{3} \mid n} \sum_{d_{4} \mid \left(d_{3}\right)^2} \sum_{d_{5} \mid \left(\frac{\left(d_{3}\right)^2}{d_{4}}\right)^2} \left(1 \cdot \mu\left(\frac{\left(\frac{\left(d_{3}\right)^2}{d_{4}}\right)^2}{d_{5}}\right)\right) \cdot \lambda\left(d_{5}\right) \cdot \left(1 \cdot \mu\left(d_{5}\right)\right) \cdot \lambda\left(d_{4}\right) \cdot \sum_{d_{6} \mid \left(d_{4}\right)^2} \left(1 \cdot \mu\left(\frac{\left(d_{4}\right)^2}{d_{6}}\right)\right) \cdot \lambda\left(d_{6}\right) \cdot \left(1 \cdot \mu\left(d_{6}\right)\right) \cdot {\frac{n}{d_{3}}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d \mid n} \sum_{d_{0} \mid d^2} \left(\psi^{3} \psi^{2} \psi^{2} \mu\right)\left(\frac{d^2}{d_{0}}\right) \cdot \lambda\left(d_{0}\right) \cdot \left(\psi^{3} \psi^{2} \psi^{2} \mu\right)\left(d_{0}\right) \cdot {\frac{n}{d}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d_{1} \mid n} \sum_{d_{2} \mid \left(d_{1}\right)^2} \left(\psi^{3} \psi^{2} \psi^{2} \mu\right)\left(\frac{\left(d_{1}\right)^2}{d_{2}}\right) \cdot \lambda\left(d_{2}\right) \cdot \left(\psi^{3} \psi^{2} \psi^{2} \mu\right)\left(d_{2}\right) \cdot {\frac{n}{d_{1}}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d \mid n} \sum_{d_{0} \mid d^2} \left(\psi^{3} \psi^{3} \mu\right)\left(\frac{d^2}{d_{0}}\right) \cdot \lambda\left(d_{0}\right) \cdot \left(\psi^{3} \psi^{3} \mu\right)\left(d_{0}\right) \cdot {\frac{n}{d}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d_{1} \mid n} \sum_{d_{2} \mid \left(d_{1}\right)^2} \left(\psi^{3} \psi^{3} \mu\right)\left(\frac{\left(d_{1}\right)^2}{d_{2}}\right) \cdot \lambda\left(d_{2}\right) \cdot \left(\psi^{3} \psi^{3} \mu\right)\left(d_{2}\right) \cdot {\frac{n}{d_{1}}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d \mid n} \sum_{d_{0} \mid d^2} \left(\psi^{4} \psi^{2} \mu\right)\left(\frac{d^2}{d_{0}}\right) \cdot \lambda\left(d_{0}\right) \cdot \left(\psi^{4} \psi^{2} \mu\right)\left(d_{0}\right) \cdot {\frac{n}{d}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d_{1} \mid n} \sum_{d_{2} \mid \left(d_{1}\right)^2} \left(\psi^{4} \psi^{2} \mu\right)\left(\frac{\left(d_{1}\right)^2}{d_{2}}\right) \cdot \lambda\left(d_{2}\right) \cdot \left(\psi^{4} \psi^{2} \mu\right)\left(d_{2}\right) \cdot {\frac{n}{d_{1}}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d \mid n} \sum_{d_{0} \mid d^2} \left(\psi^{5} \mu\right)\left(\frac{d^2}{d_{0}}\right) \cdot \lambda\left(d_{0}\right) \cdot \left(\psi^{5} \mu\right)\left(d_{0}\right) \cdot {\frac{n}{d}} = \varphi\left(n\right) </script>, <script type="math/tex; mode=display"> \sum_{d_{1} \mid n} \sum_{d_{2} \mid \left(d_{1}\right)^2} \left(\psi^{5} \mu\right)\left(\frac{\left(d_{1}\right)^2}{d_{2}}\right) \cdot \lambda\left(d_{2}\right) \cdot \left(\psi^{5} \mu\right)\left(d_{2}\right) \cdot {\frac{n}{d_{1}}} = \varphi\left(n\right) </script>]

In [16]:
identities.index(sigma + phi & tau * id)

32591